In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
import pandas as pd

def calculate_properties(mol: Chem.Mol) -> dict:
    Chem.SanitizeMol(mol)
    return {
        'HAC': mol.GetNumHeavyAtoms(),
        'MW': round(Descriptors.MolWt(mol), 2),
        'HBA': Descriptors.NumHAcceptors(mol),
        'HBD': Descriptors.NumHDonors(mol),
        'LogP': round(Descriptors.MolLogP(mol), 2),
        'RotB': Descriptors.NumRotatableBonds(mol),
        'TPSA': round(Descriptors.TPSA(mol), 2),
        'FSP3': round(Chem.rdMolDescriptors.CalcFractionCSP3(mol), 2),
        'InChIKey': Chem.MolToInchiKey(mol),
    }

# List of available reaction IDs :
# ['a1', 'a2', 'a7']

# Select reaction id 
reaction_id = 'a7'

In [5]:
path_to_reaction_file = "../synthons_example/Freedom_3_0_reactions_example.tsv"
path_to_synthons_file = "../synthons_example/Freeedom 3_0_synthons_example.tsv"

reaction_df = pd.read_csv(path_to_reaction_file, sep='\t')
synthons_df = pd.read_csv(path_to_synthons_file, sep='\t')

reaction_row = reaction_df[reaction_df['reaction_id'] == reaction_id].iloc[0]
reaction_smarts = reaction_row['Reaction']
rxn = AllChem.ReactionFromSmarts(reaction_smarts)

components = reaction_row['components']

# Get synthons by role
s1 = synthons_df[(synthons_df['reaction_id'] == reaction_id) & (synthons_df['synton#'] == 1)]
s2 = synthons_df[(synthons_df['reaction_id'] == reaction_id) & (synthons_df['synton#'] == 2)]
s3 = synthons_df[(synthons_df['reaction_id'] == reaction_id) & (synthons_df['synton#'] == 3)]

results = []

# 2-component enumeration
if components == 2:
    for _, row1 in s1.iterrows():
        for _, row2 in s2.iterrows():
            try:
                mol1 = Chem.MolFromSmiles(row1['SMILES'])
                mol2 = Chem.MolFromSmiles(row2['SMILES'])
                product_tuples = rxn.RunReactants((mol1, mol2))
                for product_tuple in product_tuples:
                    product = product_tuple[0]
                    product_smiles = Chem.MolToSmiles(product)
                    props = calculate_properties(product)
                    product_id = f"{reaction_id}_{row1['synton_id']}_{row2['synton_id']}"
                    results.append({'SMILES': product_smiles, 'ID': product_id, **props})
            except:
                continue

# 3-component enumeration
elif components == 3:
    for _, row1 in s1.iterrows():
        for _, row2 in s2.iterrows():
            for _, row3 in s3.iterrows():
                try:
                    mol1 = Chem.MolFromSmiles(row1['SMILES'])
                    mol2 = Chem.MolFromSmiles(row2['SMILES'])
                    mol3 = Chem.MolFromSmiles(row3['SMILES'])
                    product_tuples = rxn.RunReactants((mol1, mol2, mol3))
                    for product_tuple in product_tuples:
                        product = product_tuple[0]
                        product_smiles = Chem.MolToSmiles(product)
                        props = calculate_properties(product)
                        product_id = f"{reaction_id}_{row1['synton_id']}_{row2['synton_id']}_{row3['synton_id']}"
                        results.append({'SMILES': product_smiles, 'ID': product_id, **props})
                except:
                    continue


results_df = pd.DataFrame(results, columns=[
    "SMILES", "ID", "HAC", "MW", "HBA", "HBD", "LogP", "RotB", "TPSA", "FSP3", "InChIKey"
])
results_df.to_csv('enumeration_results_test.tsv', index=False, sep='\t')
